In [1]:
import torch
import random
print(torch.cuda.is_available())

True


In [2]:
genes = [0, 1, 2, 3]
ab = [0, 1, 2, 3]

data_old = [{'genes': [1, 0, 0, 0], 'res': [0, 0, 0, 0]},
        {'genes': [0, 1, 0, 0], 'res': [0, 1, 0, 0]},
        {'genes': [0, 0, 1, 0], 'res': [0, 0, 0, 0]},
        {'genes': [0, 0, 0, 1], 'res': [0, 0, 0, 0]},
        {'genes': [1, 1, 0, 0], 'res': [0, 1, 0, 0]},
        {'genes': [1, 0, 1, 0], 'res': [0, 0, 0, 0]},
        {'genes': [1, 0, 0, 1], 'res': [0, 0, 0, 0]},
        {'genes': [0, 1, 1, 0], 'res': [0, 1, 1, 0]},
        {'genes': [0, 1, 0, 1], 'res': [0, 1, 0, 1]},
        {'genes': [0, 0, 1, 1], 'res': [0, 0, 0, 0]},
        {'genes': [1, 0, 0, 0], 'res': [0, 0, 0, 0]}, 
        {'genes': [0, 0, 1, 0], 'res': [0, 0, 0, 0]},
        {'genes': [0, 0, 0, 1], 'res': [0, 0, 0, 0]},
        {'genes': [0, 1, 0, 0], 'res': [0, 1, 0, 0]},
        {'genes': [0, 1, 0, 0], 'res': [0, 1, 0, 0]},
        {'genes': [0, 1, 0, 0], 'res': [0, 1, 0, 0]},
        ]

data = [{'genes': [0, 2, 4, 6], 'res': [0, 0, 0, 0]},
        {'genes': [1, 2, 4, 6], 'res': [0, 0, 0, 0]},
        {'genes': [0, 3, 4, 6], 'res': [0, 1, 0, 0]},
        {'genes': [0, 2, 5, 6], 'res': [0, 0, 0, 0]},
        {'genes': [0, 2, 4, 7], 'res': [0, 0, 0, 0]},
        {'genes': [1, 3, 4, 6], 'res': [0, 1, 0, 0]},
        {'genes': [1, 2, 5, 6], 'res': [0, 0, 0, 0]},
        {'genes': [1, 2, 4, 7], 'res': [0, 0, 0, 0]},
        {'genes': [0, 3, 5, 6], 'res': [0, 1, 0, 0]},
        {'genes': [0, 3, 4, 7], 'res': [0, 1, 0, 0]},
        {'genes': [0, 2, 5, 7], 'res': [0, 0, 0, 0]},
        {'genes': [1, 2, 5, 7], 'res': [0, 0, 0, 0]},
        {'genes': [1, 3, 4, 7], 'res': [0, 1, 0, 0]},
        {'genes': [1, 3, 5, 6], 'res': [0, 1, 0, 0]},
        {'genes': [0, 3, 5, 7], 'res': [0, 1, 0, 0]},
        {'genes': [1, 3, 5, 7], 'res': [0, 1, 0, 0]},
        ]


data = [{'genes': [0, 0, 0, 0], 'res': [0, 0, 0, 0]},
        {'genes': [1, 0, 0, 0], 'res': [0, 0, 0, 0]},
        {'genes': [0, 1, 0, 0], 'res': [0, 1, 0, 0]},
        {'genes': [0, 0, 1, 0], 'res': [0, 0, 0, 0]},
        {'genes': [0, 0, 0, 1], 'res': [0, 0, 0, 0]},
        {'genes': [1, 1, 0, 0], 'res': [0, 1, 0, 0]},
        {'genes': [1, 0, 1, 0], 'res': [0, 0, 0, 0]},
        {'genes': [1, 0, 0, 1], 'res': [0, 0, 0, 0]},
        {'genes': [0, 1, 1, 0], 'res': [0, 1, 0, 0]},
        {'genes': [0, 1, 0, 1], 'res': [0, 1, 0, 0]},
        {'genes': [0, 0, 1, 1], 'res': [0, 0, 0, 0]},
        {'genes': [1, 0, 1, 1], 'res': [0, 0, 0, 0]},
        {'genes': [1, 1, 0, 1], 'res': [0, 1, 0, 0]},
        {'genes': [1, 1, 1, 0], 'res': [0, 1, 0, 0]},
        {'genes': [0, 1, 1, 1], 'res': [0, 1, 0, 0]},
        {'genes': [1, 1, 1, 1], 'res': [0, 1, 0, 0]},
        ]


data = [{'genes': [0, 0, 0, 0], 'res': [0, 0, 0, 0]},
        {'genes': [1, 0, 0, 0], 'res': [0, 0, 0, 0]},
        {'genes': [0, 1, 0, 0], 'res': [0, 1, 0, 0]},
        {'genes': [0, 0, 1, 0], 'res': [0, 0, 0, 0]},
        {'genes': [0, 0, 0, 1], 'res': [0, 0, 0, 0]},
        {'genes': [1, 1, 0, 0], 'res': [0, 1, 0, 0]},
        {'genes': [1, 0, 1, 0], 'res': [0, 0, 0, 0]},
        {'genes': [1, 0, 0, 1], 'res': [0, 0, 0, 0]},
        {'genes': [0, 1, 1, 0], 'res': [0, 1, 0, 0]},
        {'genes': [0, 1, 0, 1], 'res': [0, 1, 0, 1]},
        {'genes': [0, 0, 1, 1], 'res': [0, 0, 0, 0]},
        {'genes': [1, 0, 1, 1], 'res': [0, 0, 0, 0]},
        {'genes': [1, 1, 0, 1], 'res': [0, 1, 0, 1]},
        {'genes': [1, 1, 1, 0], 'res': [0, 1, 0, 0]},
        {'genes': [0, 1, 1, 1], 'res': [0, 1, 0, 1]},
        {'genes': [1, 1, 1, 1], 'res': [0, 1, 0, 1]},
        ]

In [3]:
class GabDataset(torch.utils.data.Dataset):
        def __init__(self, data):
                self.data = data


        def __len__(self):
                return len(self.data)

        def __getitem__(self, idx):
                d = self.data[idx]
                return torch.IntTensor(d['genes']), torch.LongTensor(d['res'])

In [4]:
class GabModel(torch.nn.Module):
        def __init__(self):
                super().__init__()
                self.genes = torch.IntTensor([0, 1, 2, 3]).to('cuda')
                self.hidden_dim = 32
                self.n_known = 2
                self.g_emb = torch.nn.Embedding(5, self.hidden_dim, padding_idx=4)
                self.g_emb2 = torch.nn.Embedding(5, self.hidden_dim, padding_idx=4)
                self.current_state = torch.nn.Parameter(torch.randn((1, self.hidden_dim), device=torch.device('cuda')), requires_grad=True)
                self.exist_emb = torch.nn.Embedding(3, self.hidden_dim, padding_idx=2)
                self.g_emb_fuser = torch.nn.Sequential(torch.nn.Linear(self.hidden_dim * 2, self.hidden_dim * 2),
                                                       torch.nn.ReLU(),
                                                       torch.nn.Linear(self.hidden_dim * 2, self.hidden_dim))
                self.loss_pred = torch.nn.Sequential(torch.nn.Linear(self.hidden_dim, self.hidden_dim // 2),
                                                  torch.nn.ReLU(),
                                                  torch.nn.Linear(self.hidden_dim // 2, self.hidden_dim // 4),
                                                  torch.nn.ReLU(),
                                                  torch.nn.Linear(self.hidden_dim // 4, 4))
                self.layers = torch.nn.Sequential(torch.nn.Linear(self.hidden_dim, self.hidden_dim * 2),
                                                  torch.nn.ReLU(),
                                                  torch.nn.Linear(self.hidden_dim * 2, self.hidden_dim * 4),
                                                  torch.nn.ReLU(),
                                                  torch.nn.Linear(self.hidden_dim * 4, self.hidden_dim * 8),
                                                  torch.nn.ReLU())
                encoder_layer = torch.nn.TransformerEncoderLayer(d_model=self.hidden_dim, nhead=4, dim_feedforward=4*self.hidden_dim, batch_first=True)
                self.transf = torch.nn.TransformerEncoder(encoder_layer, num_layers=4, enable_nested_tensor=False)
                self.head1 = torch.nn.Linear(self.hidden_dim, 2)
                self.head2 = torch.nn.Linear(self.hidden_dim, 2)
                self.head3 = torch.nn.Linear(self.hidden_dim, 2)
                self.head4 = torch.nn.Linear(self.hidden_dim, 2)

                self.exploration_chance = 1
        
        def forward(self, x):
                b = x.shape[0]
                g = self.genes.repeat(b, 1)

                exploration = random.random()
                if exploration < self.exploration_chance:
                    random_gene = torch.multinomial(torch.Tensor([1]*len(self.genes)).to('cuda'), self.n_known)
                    #print("Exploration!")
                    exploring = True
                else:
                    #print("Exploiting!")
                    exploring = False
                state = self.current_state
                best_gene = torch.zeros(self.n_known, dtype=torch.int32, device=torch.device('cuda'))
                all_loss_preds = []
                for j in range(self.n_known):
                    predicted_loss = self.loss_pred(state)
                    all_loss_preds.append(predicted_loss)
                    if exploring:
                        gene = random_gene[j].repeat(b, 1).view(b)
                    else:
                        min_loss, gene = predicted_loss.min(dim=1)
                        best_gene[j] = gene
                    gemb = self.g_emb2(gene)
                    state = self.g_emb_fuser(torch.cat([state, gemb], dim=1))
                
                if exploring:
                    sel_gene = random_gene
                else:
                    sel_gene = best_gene
                final_loss_pred = predicted_loss[:,gene]
                    
                

                #predicted_loss_random_gene = predicted_loss[:,random_gene]
                sel_genes = g[:,sel_gene]
                sel_known = x[:,sel_gene]
                sel_g_emb = self.g_emb(sel_genes)
                sel_exist_emb = self.exist_emb(sel_known)
                sel_emb = sel_g_emb + sel_exist_emb
                #sel_genes = x[:,random_gene]
                #sel_emb = self.g_emb(sel_genes)
                out_emb = self.transf(sel_emb)
                
                out_mean = torch.mean(out_emb, dim=1)

                #predicted_loss = self.loss_pred(self.current_state)
                #predicted_loss_r_gene = predicted_loss[:,random_gene]    
            
                pred_1 = self.head1(out_mean)
                pred_2 = self.head2(out_mean)
                pred_3 = self.head3(out_mean)
                pred_4 = self.head4(out_mean)
                
                return pred_1, pred_2, pred_3, pred_4, all_loss_preds, sel_gene, final_loss_pred

In [5]:
def compute_loss(pred_1, pred_2, pred_3, pred_4, labels_1, labels_2, labels_3, labels_4, final_loss_pred):
        loss_fn = torch.nn.CrossEntropyLoss()
        rl_loss_fn = torch.nn.SmoothL1Loss()
        loss_1 = loss_fn(pred_1, labels_1)
        loss_2 = loss_fn(pred_2, labels_2)
        loss_3 = loss_fn(pred_3, labels_3)
        loss_4 = loss_fn(pred_4, labels_4)
        
        total_pred_loss = (loss_1 + loss_2 + loss_3 + loss_4).mean()
        #print("Mean CE loss: {}".format(total_pred_loss))
        #print(predicted_loss.shape)
        #print(total_pred_loss.shape)
        total_rl_loss = rl_loss_fn(final_loss_pred.mean().view(-1, 1), total_pred_loss.view(-1,1)).mean()
        total_loss = total_pred_loss + 0.5*total_rl_loss
        return total_loss, total_rl_loss

In [6]:
dataset = GabDataset(data)
bs = 1
data_loader = torch.utils.data.DataLoader(dataset, batch_size=bs)

model = GabModel()
model.to('cuda')

optimizer = torch.optim.AdamW(model.parameters(),
                                  lr=1e-3,
                                  weight_decay=1e-6)

epochs = 100

print_every_n_epochs = 10

for e in range(epochs):
        n_correct_ab_1 = 0
        n_ab_1 = 0
        n_correct_ab_2 = 0
        n_ab_2 = 0
        n_correct_ab_3 = 0
        n_ab_3 = 0
        n_correct_ab_4 = 0
        n_ab_4 = 0
        for d, l in data_loader:
                d = d.to('cuda')
                l = l.to('cuda')
                labels_1 = l[:,0]
                labels_2 = l[:,1]
                labels_3 = l[:,2]
                labels_4 = l[:,3]
                pred_1, pred_2, pred_3, pred_4, all_loss_preds, sel_gene, final_loss_pred = model(d)
                loss, rl_loss = compute_loss(pred_1, pred_2, pred_3, pred_4, labels_1, labels_2, labels_3, labels_4, final_loss_pred)
                if (e + 1) % print_every_n_epochs == 0:
                        print("Loss is: {}".format(loss))
                        print("RL loss is: {}".format(rl_loss))
                        print("Selected genes are: {}".format(sel_gene))
                        print("Predicted loss for final genes are: {}".format(final_loss_pred))
                        print("Predicted loss for all genes are: {}".format(all_loss_preds))
                hard_pred_1 = torch.argmax(pred_1, dim=1)
                hard_pred_2 = torch.argmax(pred_2, dim=1)
                hard_pred_3 = torch.argmax(pred_3, dim=1)
                hard_pred_4 = torch.argmax(pred_4, dim=1)
                
                n_correct_ab_1 += (hard_pred_1 == labels_1).sum()
                n_ab_1 += bs
                n_correct_ab_2 += (hard_pred_2 == labels_2).sum()
                n_ab_2 += bs
                n_correct_ab_3 += (hard_pred_3 == labels_3).sum()
                n_ab_3 += bs
                n_correct_ab_4 += (hard_pred_4 == labels_4).sum()
                n_ab_4 += bs
                
                
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
        model.exploration_chance = max(model.exploration_chance - 0.01, 0)
        acc_ab_1 = n_correct_ab_1 / n_ab_1
        acc_ab_2 = n_correct_ab_2 / n_ab_2
        acc_ab_3 = n_correct_ab_3 / n_ab_3
        acc_ab_4 = n_correct_ab_4 / n_ab_4
        mean_acc = (acc_ab_1 + acc_ab_2 + acc_ab_3 + acc_ab_4) / 4
        #print("--- Epoch {} --- Accuracy AB 0: {} ---".format(e, acc_ab_1))
        print("--- Epoch {} --- Accuracy AB 1: {} ---".format(e, acc_ab_2))
        #print("--- Epoch {} --- Accuracy AB 2: {} ---".format(e, acc_ab_3))
        #print("--- Epoch {} --- Accuracy AB 3: {} ---".format(e, acc_ab_4))
        print("--- Epoch {} --- Mean Accuracy: {} ---".format(e, mean_acc))

#print("Final Loss is: {}".format(loss))
#print("Final Selected genes are: {}".format(out_i))
#print("Final Selected gene scores are: {}".format(scores))
#print("Final Excluded gene scores scores are: {}".format(scores_comp))
#print("Final All gene scores scores are: {}".format(all_scores))

--- Epoch 0 --- Accuracy AB 1: 0.5625 ---
--- Epoch 0 --- Mean Accuracy: 0.78125 ---
--- Epoch 1 --- Accuracy AB 1: 0.4375 ---
--- Epoch 1 --- Mean Accuracy: 0.8125 ---
--- Epoch 2 --- Accuracy AB 1: 0.4375 ---
--- Epoch 2 --- Mean Accuracy: 0.796875 ---
--- Epoch 3 --- Accuracy AB 1: 0.6875 ---
--- Epoch 3 --- Mean Accuracy: 0.875 ---
--- Epoch 4 --- Accuracy AB 1: 0.6875 ---
--- Epoch 4 --- Mean Accuracy: 0.84375 ---
--- Epoch 5 --- Accuracy AB 1: 0.6875 ---
--- Epoch 5 --- Mean Accuracy: 0.875 ---
--- Epoch 6 --- Accuracy AB 1: 0.5625 ---
--- Epoch 6 --- Mean Accuracy: 0.828125 ---
--- Epoch 7 --- Accuracy AB 1: 0.5625 ---
--- Epoch 7 --- Mean Accuracy: 0.84375 ---
--- Epoch 8 --- Accuracy AB 1: 0.75 ---
--- Epoch 8 --- Mean Accuracy: 0.84375 ---
Loss is: 1.1051157712936401
RL loss is: 0.49711084365844727
Selected genes are: tensor([2, 0], device='cuda:0')
Predicted loss for final genes are: tensor([[-0.1405]], device='cuda:0', grad_fn=<IndexBackward0>)
Predicted loss for all genes 